## 图说Batch Normalization

https://aibyhand.substack.com/p/9-can-you-calculate-batch-normalization

下面是关于批量归一化（Batch Normalization）图例的描述的：

[1] 给定
 一个包含4个训练样本的小批量数据，每个样本有3个特征。

[2] 线性层
 通过与权重和偏置相乘来获得新的特征。

[3] ReLU
 应用ReLU激活函数，其效果是抑制负值。在这个练习中，-2 被设置为 0。

[4] 批量统计
 计算这个小批量数据中四个样本的总和、平均值、方差和标准差。
 注意这些统计量是针对每一行（即，每个特征维度）计算的。

[5] 移至均值 = 0
 从每个训练样本的激活值中减去均值（绿色）
 预期的效果是让每个维度中的4个激活值的平均值为零。

[6] 缩放至方差 = 1
 除以标准差（橙色）
 预期的效果是让这4个激活值具有等于一的方差。

[7] 缩放与平移
 将步骤[6]中得到的标准化特征乘以一个线性变换矩阵，并将结果传递给下一层。
 预期的效果是对标准化特征值进行缩放和平移，以获得一个新的均值和方差，这些是由网络学习得到的。
 对角线元素和最后一列是网络将要学习的可训练参数。

In [1]:
from IPython.display import Image
Image(url= "bn0.png")

## 批量归一化（Batch Normalization，简称BN）原理

批量归一化（Batch Normalization，简称BN）是一种用于加速深度神经网络训练的技术，由Sergey Ioffe和Christian Szegedy在2015年提出。其主要目的是减少内部协变量偏移（internal covariate shift），使得网络能够更快地收敛，并且有助于减轻过拟合的问题。

以下是批量归一化的详细介绍：

### 基本概念

批量归一化是在神经网络的隐藏层之间插入的一种规范化技术，它通过对网络每一层的输入进行规范化处理，确保这些输入具有零均值和单位方差。这种处理可以改善训练过程中的梯度流，并且有助于使每一层的学习任务更加稳定和简单。

### 工作原理

批量归一化的工作原理如下：

1. **计算统计数据**：
   - 在训练过程中，对于每一个mini-batch的数据，BN会计算该批次数据的均值和方差。
   - 这些统计数据是基于当前mini-batch中的所有样本计算得出的。

2. **标准化**：
   - 使用上述计算出的均值和方差对mini-batch中的数据进行标准化，使其具有零均值和单位方差。
   - 标准化公式可以表示为：$$ \hat{x}^{(k)} = \frac{x^{(k)} - \mu^{(k)}}{\sqrt{\sigma^{(2)(k)} + \epsilon}} $$，其中 $ x^{(k)} $ 是第k个样本，$ \mu^{(k)} $ 和 $ \sigma^{(2)(k)} $ 分别是均值和方差，$ \epsilon $ 是一个小常数用来防止除以零的情况发生。

3. **缩放和平移**：
   - 在标准化之后，BN允许对数据进行缩放和平移，以便模型可以学习到适当的分布。
   - 这是通过两个可学习的参数 γ（scale）和 β（shift）实现的：$$ y^{(k)} = \gamma \hat{x}^{(k)} + \beta $$。

### 优点

- **加速训练**：BN减少了内部协变量偏移，从而加快了训练速度。
- **正则化效果**：BN有一定的正则化作用，可以帮助减少过拟合的风险。
- **更宽松的初始化要求**：使用BN后，模型对权重的初始值更加不敏感。
- **可能允许更高的学习率**：由于BN稳定了训练过程，因此可以使用更高的学习率。

### 测试阶段

在测试阶段，我们不再使用mini-batch的统计数据来进行归一化，而是使用整个训练集的移动平均统计值来替代，这样可以更好地反映整个数据集的特性。

需要注意的是，尽管BN在很多情况下表现出了显著的优势，但它并不是万能的解决方案，在某些特定的情况下可能不会带来明显的改进。此外，BN对某些类型的模型，如RNN，可能不如对CNN那么有效。

## Batch Normalization在PyTorch中的实现

### nn.BatchNorm基本使用方法

&emsp;&emsp;在PyTorch中，我们使用nn.Linear构建线性层，类似的，我们通过使用nn.BatchNorm类来构建BN层，进而可以进行Batch Normalization归一化操作，并且在后续操作过程中我们会发现，BN层和线性层之间由诸多相似之处。同时，出于对不同类型数据的不同处理需求，nn.BatchNorm1d类主要用于处理2d数据，nn.BatchNorm2d则主要用来处理3d数据。简单来说，目前我们所使用的面板数据（二维表格数据）都属于2d数据，使用nn.BatchNorm1d处理即可。而后续在处理图像数据时，则需要视情况和使用nn.BatchNorm2d类。

In [ ]:
# 查看帮助
#nn.BatchNorm1d

首先我们解释其中三个核心参数：

- num_features：输入数据的特征数量（假设为n），也就是前一层神经元数量或原始数据集特征数量，根据此前的论述，BN层最终构建的是一个n*n的对角矩阵，对角线元素包含$\gamma$，并且截距项为$\beta$；
- eps：方差分母修正项，为了防止分母为0，一般取值为1e-5，也就是类默认值；
- affine：是否进行仿射变换，需要注意的是，此时进行仿射变换时将使用无偏估计进行期望和方差的计算，并且初始条件下$\gamma=1，\beta=0$，当参数取值为True时，会显式设置$\gamma和\beta$参数并带入进行梯度下降迭代计算，取值为False时，参数不显示，实际的数据归一化过程就是对原数据进行无偏估计下的Z-Score变换。

简单进行尝试

In [1]:
import torch
import torch.nn as nn

In [2]:
f = torch.arange(9).reshape(3, 3).float()
f

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [3]:
# 实例化一个BN类
bn1 = nn.BatchNorm1d(3)

In [4]:
bn1(f)

tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]], grad_fn=<NativeBatchNormBackward0>)

In [5]:
f

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [11]:
# Z-Score计算结果
(f - torch.mean(f, 0)) / torch.std(f, 0)

tensor([[-1., -1., -1.],
        [ 0.,  0.,  0.],
        [ 1.,  1.,  1.]])

In [6]:
# 样本方差计算结果
(f - torch.mean(f, 0)) / torch.std(f, 0,unbiased=False)

tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]])

> pytorch中默认的标准差计算是样本无偏估计，而BN再进行归一化计算时实际上采用的是样本方差，二者区别如下，样本标准差为：$$std(x) =  \sqrt\frac{\sum^n_{i=1}(x_i-\bar x)^2}{n}$$但对样本整体方差的无偏估计为：$$std_{unbiased}(x) = \sqrt \frac{\sum^n_{i=1}(x_i-\bar x)^2}{n-1}$$

&emsp;&emsp;当然，根据此前的介绍，BN层和线性层有许多类似的地方，这里我们可以查看BN层的参数情况。

In [7]:
list(bn1.parameters())

[Parameter containing:
 tensor([1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], requires_grad=True)]

据此也可确认在迭代开始之前，$\gamma和\beta$的初始值。同时，根据参数的可微性，我们也能看出其最终也是需要通过反向传播计算梯度，然后利用梯度下降进行求解的。当然，此处如果我们将affine参数设置为False，则无法查看参数，并且归一化过程就是简单的在无偏估计下对数据进行Z-Score归一化。

In [77]:
bn2 = nn.BatchNorm1d(3, affine=False)

In [78]:
list(bn2.parameters())       # 此时无法查看bn2参数

[]

In [79]:
bn2(f)                       # 处理后的数据也不是可微的，无法进行反向传播，因此也无法修改BN参数

tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]])

### 训练数据均值与方差的迭代计算

* 训练阶段的统计信息更新

在训练阶段，对于每一个mini-batch，BN层都会计算出该mini-batch的均值和方差，并使用这些统计信息来归一化数据。同时，BN层也会更新整个训练集的均值和方差的估计值。这一过程通常通过移动平均（moving average）的方式来实现。

* 计算mini-batch的统计信息

对于一个mini-batch中的数据，假设mini-batch的大小为$ m $，特征维度为$ n $，那么mini-batch中的数据可以表示为 $ X = \{x_{ij}\}_{m \times n} $，其中 $ x_{ij} $ 表示第 $ i $ 个样本的第 $ j $ 个特征。

* 更新累计统计信息

在训练过程中，BN层会使用一个移动平均来更新整个训练集的统计信息。移动平均公式如下：

1. **更新均值的移动平均**：

$$ \mu_{\text{moving}} \leftarrow \alpha \cdot \mu_{\text{moving}} + (1 - \alpha) \cdot \mu $$

其中 $ \alpha $ 是一个接近1的衰减因子（通常取值为0.9或0.999），$ \mu $ 是当前mini-batch的均值。

2. **更新方差的移动平均**：

$$ \sigma_{\text{moving}}^2 \leftarrow \alpha \cdot \sigma_{\text{moving}}^2 + (1 - \alpha) \cdot \sigma^2 $$

其中 $ \sigma^2 $ 是当前mini-batch的方差。

* 测试阶段使用累积统计信息

在测试阶段，BN层会使用整个训练过程中累积的统计信息来进行归一化。这意味着在测试时，BN层会使用上述计算得到的 $ \mu_{\text{moving}} $ 和 $ \sigma_{\text{moving}}^2 $ 来归一化输入数据。

* 实际应用中的实现

在PyTorch中，BN层会自动处理这些计算和更新。你可以通过查看 `running_mean` 和 `running_var` 属性来了解累积的统计信息：

```python
# 获取BN层的累积统计信息
running_mean = bn_layer.running_mean
running_var = bn_layer.running_var
```

在训练过程中，这些属性会被自动更新。在测试阶段，BN层会使用这些累积的统计信息来进行归一化。

* 总结

通过使用移动平均来更新整个训练集的统计信息，BN层能够在训练过程中不断调整归一化的参数，并在测试阶段使用更准确的统计信息来进行归一化，从而提高模型的稳定性和泛化能力。

&emsp;&emsp;

In [102]:
f = torch.arange(9).reshape(3, 3).float()
f

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [77]:
bn1 = nn.BatchNorm1d(3)

In [18]:
#help(bn1)

In [13]:
bn1.running_mean
#bn1.running_var

tensor([0., 0., 0.])

In [14]:
bn1(f)

tensor([[-1.2247, -1.2247, -1.2247],
        [ 0.0000,  0.0000,  0.0000],
        [ 1.2247,  1.2247,  1.2247]], grad_fn=<NativeBatchNormBackward0>)

In [15]:
bn1.running_mean

tensor([0.3000, 0.4000, 0.5000])

In [16]:
f.mean(0)

tensor([3., 4., 5.])

In [87]:
bn1.running_mean

tensor([0.3000, 0.4000, 0.5000])

In [81]:
bn3 = nn.BatchNorm1d(3)

In [83]:
list(bn3.parameters())

[Parameter containing:
 tensor([1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], requires_grad=True)]

In [73]:
print(bn3.running_mean)
bn3(f)
print(bn3.running_mean)

tensor([2.9453, 3.9270, 4.9088])
tensor([2.9507, 3.9343, 4.9179])


In [34]:
0*0.9+0.1*3

0.30000000000000004

In [36]:
0.3*0.9+0.1*3

0.5700000000000001

In [33]:
print(bn3.running_mean)
bn3(f)
print(bn3.running_mean)

tensor([0.3000, 0.4000, 0.5000])
tensor([0.5700, 0.7600, 0.9500])


In [15]:
torch.mean(f, 0)

tensor([3., 4., 5.])

In [19]:
bn3.running_mean * 0.9 + 0.1 * torch.mean(f, 0)

NameError: name 'bn3' is not defined

In [14]:
bn3.running_var * 0.9 + 0.1 * torch.var(f, 0)

tensor([1.8000, 1.8000, 1.8000])

当然，在这种设置下，如果同一批数据，当迭代很多轮之后，running_mean/var就会最终取到sample_mean/var。

能够发现，BN记录结果朝向整体真实结果靠拢。

- 迭代记录的原因

&emsp;&emsp;至于为什么神经网络为什么采用迭代方法求解训练集特征的均值和方差，而不是直接直接按列进行统计。BN通过迭代求解的主要原因有两点，其一是面对超大规模数据时候求解均值或方差可能无法直接进行求解，尤其是如果数据是分布式存储的话一次性求解均值和方差就会更加困难；其二，迭代计算是深度学习求解参数的基本过程，所有深度学习框架为了更好地满足迭代计算，都进行了许多精巧的设计，从而使得迭代计算更加高效和平稳，而在计算训练集整体均值时，如果借助神经网络向前传播的过程进行迭代记录，便能够在控制边际成本的情况下快速达成目标。

- momentum参数设置

&emsp;&emsp;关于momentum的设置，一般来说，为了尽可能获取到更加准确的训练集整体统计量，当每一个小批数据数据量比较小时，我们应该将历史数据比重调高，也就是降低momentum取值，以减少局部规律对获取总体规律的影响，当然，此时我们也需要增加遍历数据的次数epochs；而反之则可以考虑增大momentum取值。

- track_running_stats参数

&emsp;&emsp;对于BN来说，最后一个参数就是track_running_stats，默认取值为True。当此参数为True时，BN层会在每次迭代过程中会结合历史记录更新running_mean/var，当track_running_stats=False时，BN层将running_mean/var为None，并且在进行预测时会根据输入的小批数据进行均值和方差计算。在大多数情况下，并不推荐修改该参数的默认取值。

#### classwork 1

* 设定一个4个特征5个样本的张量，并使用BN层进行归一化，并对比通过torch.mean的方式收到归一化的结果差异
* 输出此bn层的参数和running_mean/running_var
* 修改momentum参数，并观察结果
* 通过循环运行50次归一，对比其running_mean/running_var与torch.mean的结果差异

In [84]:
f = torch.arange(20).reshape(5, 4).float()
f

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.],
        [16., 17., 18., 19.]])

In [97]:
bn4= nn.BatchNorm1d(4, momentum=0.3)

In [93]:
bn4(f)

tensor([[-1.4142, -1.4142, -1.4142, -1.4142],
        [-0.7071, -0.7071, -0.7071, -0.7071],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.7071,  0.7071,  0.7071,  0.7071],
        [ 1.4142,  1.4142,  1.4142,  1.4142]],
       grad_fn=<NativeBatchNormBackward0>)

In [98]:
for i in range(50):
    bn4(f)
print(bn4.running_mean)

tensor([ 8.0000,  9.0000, 10.0000, 11.0000])


In [89]:
list(bn4.parameters())

[Parameter containing:
 tensor([1., 1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], requires_grad=True)]

In [96]:
f.mean(0)

tensor([ 8.,  9., 10., 11.])

In [94]:
bn4.running_mean

tensor([2.4000, 2.7000, 3.0000, 3.3000])

In [95]:
bn4.running_var

tensor([12.7000, 12.7000, 12.7000, 12.7000])

In [88]:
(f-f.mean(0))/f.std(0,unbiased=False)

tensor([[-1.4142, -1.4142, -1.4142, -1.4142],
        [-0.7071, -0.7071, -0.7071, -0.7071],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.7071,  0.7071,  0.7071,  0.7071],
        [ 1.4142,  1.4142,  1.4142,  1.4142]])

## BN层在神经网络中的应用

###  model.train()与model.eval()

- 伴随向前传播调整running_mean/var

&emsp;&emsp;尽管BN找到了能够计算训练集整体均值和方差的方法，但截至目前，还是没有解决如何将在训练过程中记录的running_mean/var应用到测试集当中的方法。核心原因在于，BN层的running_mean/var是伴随向前传播同步调整的。也就是说，在track_running_stats开启时，只要进行一次向前传播，就会更新一次running_mean/var，这明显是不合适的（因为我们不能根据测试集数据修改参数）。但是我们也知道，只要要进行测试集的测试，就一定需要模型执行测试数据的向前传播以得出模型结果，此时应该怎么办呢？

- model.train()与model.eval()

&emsp;&emsp;此时就要用到PyTorch中适用于nn模块中所有模型的一种方法——model.train()与model.eval()。其中，model.train()表示开启模型训练模式，在默认情况下，我们实例化的每一个模型都是出于训练模式的，而model.eval()则表示将模型转化为测试模式。我们可以简单查看该方法的相关说明。

In [99]:
bn5 = nn.BatchNorm1d(3)

In [100]:
bn5.train()

BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

当然，开始训练模式时参数取值为False则代表开启测试模式，即bn5.train(False)和bn5.eval()等效。

&emsp;&emsp;那么，训练模式和测试模式的根本区别是什么？简单来说就是，在PyTorch中，其实有很多类似BN类一样，会在向前传播过程中直接自动修改模型参数，而当模型出于测试模式时，会避免这种情况发生，即模型出于测试模式时不会根据当前输入数据情况调整running_mean/var。并且，模型出于测试模式时，BN层会利用已经记录下的running_mean/var对数据进行归一化。

> 在PyTorch中，训练模式和测试模式的区分，相当于是给模型提供了可设置的两种行为模式。

In [103]:
f

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [104]:
bn5 = nn.BatchNorm1d(3)
bn5.train() # 进入训练模式

BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [105]:
print(bn5.running_mean)
bn5(f)
print(bn5.running_mean)
#bn5.running_var

tensor([0., 0., 0.])
tensor([0.3000, 0.4000, 0.5000])


In [114]:
bn5.eval()

BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [108]:
bn5.train()

BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [121]:
print(bn5.running_mean)
bn5(f)
print(bn5.running_mean)               

tensor([1.4057, 1.8742, 2.3428])
tensor([1.4057, 1.8742, 2.3428])


In [109]:
bn5.training

True

另外我们需要知道，当模型（model）由多个模块（module）共同集成时，各模块的状态由模型整体状态决定。

### 利用nn.BatchNorm构建带BN的神经网络

&emsp;&emsp;有了对nn.BatchNorm类基本了解之后，我们来尝试构建带有BN层的神经网络模型。根据上一节的介绍，我们知道BN本质上是一种自适应的数据分布调整算法，同时我们也知道，数据分布的调整并不影响，因此我们可以在任何需要的位置都进行BN归一化。另外，根据Glorot条件，我们知道在模型构建过程中需要力求各梯度计算的有效性和平稳性，因此我们可以考虑在每一个线性层前面或者后面进行数据归一化处理。

&emsp;&emsp;在实际模型构建过程中，在模型中添加BN层和添加线性层类似，我们只需要在自定义模型类的init方法中添加BN层，并在向前传播方法中确定BN层的调用方法即可。而在具体BN层位置选择方面，是放在线性层前面还是放在线性层后面，目前业内没有定论，既没有理论论证BN层的最佳位置，也没有严谨的实验证明在哪个位置放置BN层效果最好。因此我们大可根据自己的习惯，以及自身经验的判断选择BN层的位置。当然，为了实验方便，我们会同时创建可以前置或者后置BN层的模型。（这里我们将放在隐藏层前面（也就是线性层后面）称为BN层前置，放在隐藏层后面称为BN层后置。）

In [122]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# 定义一个简单的带BN的神经网络模型
class SimpleNetWithBN(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=20, output_dim=2):
        super(SimpleNetWithBN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)  # 添加BN层
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)  # 添加BN层
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # 先通过线性层再通过BN层
        x = torch.relu(self.bn2(self.fc2(x)))  # 同样，先线性层再BN层
        x = self.fc3(x)
        x=torch.softmax(x,dim=1)
        return x

In [123]:
# 生成模拟数据
def generate_data(num_samples, input_dim, output_dim):
    input_data = torch.randn(num_samples, input_dim)  # 生成输入数据
    target_data = torch.randint(low=0, high=output_dim, size=(num_samples,))  # 生成目标标签
    return input_data, target_data

In [124]:
# 参数设置
num_samples = 100
input_dim = 10
hidden_dim = 20
output_dim = 3

# 生成模拟数据
input_data, target_data = generate_data(num_samples, input_dim, output_dim)

# 使用sklearn的train_test_split函数分割数据集
train_inputs, test_inputs, train_targets, test_targets = train_test_split(
    input_data.numpy(), target_data.numpy(), test_size=0.3, random_state=42
)

# 将numpy数组转换回PyTorch张量
train_inputs = torch.from_numpy(train_inputs).float()
test_inputs = torch.from_numpy(test_inputs).float()
train_targets = torch.from_numpy(train_targets).long()
test_targets = torch.from_numpy(test_targets).long()

# 创建TensorDataset
train_data = TensorDataset(train_inputs, train_targets)
test_data = TensorDataset(test_inputs, test_targets)

# 创建DataLoader
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [127]:
target_data

tensor([1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
        0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 1])

In [125]:


# 实例化模型
model = SimpleNetWithBN(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    for inputs, targets in train_loader:
        # 前向传播
        outputs = model(inputs)
        
        # 计算损失
        loss = criterion(outputs, targets)
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        
        # 更新权重
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 1.0679
Epoch [2/10], Loss: 1.0736
Epoch [3/10], Loss: 1.0769
Epoch [4/10], Loss: 1.1231
Epoch [5/10], Loss: 1.0656
Epoch [6/10], Loss: 1.0681
Epoch [7/10], Loss: 1.1423
Epoch [8/10], Loss: 1.0976
Epoch [9/10], Loss: 1.1334
Epoch [10/10], Loss: 1.0872


In [129]:
# 测试模型
model.eval()  # 设置模型为评估模式
with torch.no_grad():  # 不需要计算梯度
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'Accuracy of the model on the test: {100 * correct / total:.2f}%')

Accuracy of the model on the test: 23.33%
